<table style="width:100%">
<tr>
<td style="vertical-align:middle; text-align:left;">
<font size="2">
Supplementary code for the <a href="http://mng.bz/orYv">Build a Large Language Model From Scratch</a> book by <a href="https://sebastianraschka.com">Sebastian Raschka</a><br>
<br>Code repository: <a href="https://github.com/rasbt/LLMs-from-scratch">https://github.com/rasbt/LLMs-from-scratch</a>
<br>汉化的库: <a href="https://github.com/GoatCsu/CN-LLMs-from-scratch.git">https://github.com/GoatCsu/CN-LLMs-from-scratch.git</a>
</font>
</td>
<td style="vertical-align:middle; text-align:left;">
<a href="http://mng.bz/orYv"><img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/cover-small.webp" width="100px"></a>
</td>
</tr>
</table>


- 通过取消注释并运行以下单元格来安装此Notebook所需的额外软件包：

In [1]:
# pip install -r requirements-extra.txt

# 比较各种字节对编码（BPE）实现

## 使用来自 `tiktoken` 的BPE

In [2]:
from importlib.metadata import version

print("tiktoken version:", version("tiktoken"))

tiktoken version: 0.11.0


In [3]:
import tiktoken

tik_tokenizer = tiktoken.get_encoding("gpt2")

text = "Hello, world. Is this-- a test?"

In [4]:
integers = tik_tokenizer.encode(text, allowed_special={"<|endoftext|>"})

print(integers)

[15496, 11, 995, 13, 1148, 428, 438, 257, 1332, 30]


In [5]:
strings = tik_tokenizer.decode(integers)

print(strings)

Hello, world. Is this-- a test?


In [6]:
print(tik_tokenizer.n_vocab)

50257


<br>
&nbsp;

## 使用GPT-2中使用的原始BPE实现

In [8]:
import os
import urllib.request

if not os.path.exists("bpe_openai_gpt2.py"):
    url = ("https://raw.githubusercontent.com/rasbt/"
           "LLMs-from-scratch/main/ch02/02_bonus_bytepair-encoder/"
           "bpe_openai_gpt2.py")
    file_path = "bpe_openai_gpt2.py"
    urllib.request.urlretrieve(url, file_path)

In [9]:
from bpe_openai_gpt2 import get_encoder, download_vocab

In [10]:
download_vocab()

Fetching encoder.json: 1.04Mit [00:00, 2.31Mit/s]                                                   
Fetching vocab.bpe: 457kit [00:00, 1.06Mit/s]                                                       


In [11]:
orig_tokenizer = get_encoder(model_name="gpt2_model", models_dir=".")

In [12]:
integers = orig_tokenizer.encode(text)

print(integers)

[15496, 11, 995, 13, 1148, 428, 438, 257, 1332, 30]


In [13]:
strings = orig_tokenizer.decode(integers)

print(strings)

Hello, world. Is this-- a test?


<br>
&nbsp;

## 通过Hugging Face transformers使用BPE

In [14]:
import transformers

transformers.__version__

'4.55.4'

In [15]:
from transformers import GPT2Tokenizer

hf_tokenizer = GPT2Tokenizer.from_pretrained("gpt2")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

In [16]:
hf_tokenizer(strings)["input_ids"]

[15496, 11, 995, 13, 1148, 428, 438, 257, 1332, 30]

In [17]:
from transformers import GPT2TokenizerFast

hf_tokenizer_fast = GPT2TokenizerFast.from_pretrained("gpt2")

In [18]:

hf_tokenizer_fast(strings)["input_ids"]

[15496, 11, 995, 13, 1148, 428, 438, 257, 1332, 30]

##Using my own from-scratch BPE tokenizer

In [ ]:
import os
import sys
import io
import nbformat
import types

def import_from_notebook():
    def import_definitions_from_notebook(fullname, names):
        current_dir = os.getcwd()
        path = os.path.join(current_dir, "..", "05_bpe-from-scratch", fullname + ".ipynb")
        path = os.path.normpath(path)

        # Load the notebook
        if not os.path.exists(path):
            raise FileNotFoundError(f"Notebook file not found at: {path}")

        with io.open(path, "r", encoding="utf-8") as f:
            nb = nbformat.read(f, as_version=4)

        # Create a module to store the imported functions and classes
        mod = types.ModuleType(fullname)
        sys.modules[fullname] = mod

        # Go through the notebook cells and only execute function or class definitions
        for cell in nb.cells:
            if cell.cell_type == "code":
                cell_code = cell.source
                for name in names:
                    # Check for function or class definitions
                    if f"def {name}" in cell_code or f"class {name}" in cell_code:
                        exec(cell_code, mod.__dict__)
        return mod

    fullname = "bpe-from-scratch"
    names = ["BPETokenizerSimple"]

    return import_definitions_from_notebook(fullname, names)

In [ ]:
imported_module = import_from_notebook()
BPETokenizerSimple = getattr(imported_module, "BPETokenizerSimple", None)

tokenizer_gpt2 = BPETokenizerSimple()
tokenizer_gpt2.load_vocab_and_merges_from_openai(
    vocab_path=os.path.join("gpt2_model", "encoder.json"),
    bpe_merges_path=os.path.join("gpt2_model", "vocab.bpe")
)

In [ ]:
integers = tokenizer_gpt2.encode(text)

print(integers)

<br>
&nbsp;

## 快速性能基准测试

In [20]:
import os##导入os库
import urllib.request ##导入request库

if not os.path.exists("the-verdict.txt"):##如果文件不存在则创建，防止因文件已存在而报错
    url = ("https://raw.githubusercontent.com/rasbt/"
           "LLMs-from-scratch/main/ch02/01_main-chapter-code/"
           "the-verdict.txt")
    file_path = "the-verdict.txt"
    urllib.request.urlretrieve(url, file_path)##从指定的地点读取文件

In [22]:
# with open('../01_main-chapter-code/the-verdict.txt', 'r', encoding='utf-8') as f:
#     raw_text = f.read()

with open('the-verdict.txt', 'r', encoding='utf-8') as f:
    raw_text = f.read()

### Original OpenAI GPT-2 tokenizer

In [ ]:
%timeit orig_tokenizer.encode(raw_text)

### Tiktoken OpenAI GPT-2 tokenizer

In [ ]:
%timeit tik_tokenizer.encode(raw_text)

### Hugging Face OpenAI GPT-2 tokenizer

In [ ]:
%timeit hf_tokenizer(raw_text)["input_ids"]

In [ ]:
%timeit hf_tokenizer(raw_text, max_length=5145, truncation=True)["input_ids"]

In [ ]:
%timeit hf_tokenizer_fast(raw_text)["input_ids"]

In [ ]:
%timeit hf_tokenizer_fast(raw_text, max_length=5145, truncation=True)["input_ids"]

### My own GPT-2 tokenizer (for educational purposes)

In [ ]:
%timeit tokenizer_gpt2.encode(raw_text)